# TO DO
- clean up, possibly consolidate table creation all into this file
- do some pre-filtering on sites that are in the OPP-115 corpus vs. those that are not

In [13]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import numpy as np

## Intro
This file loads and persists data from the OPP-115 corpus, courtesy of UsablePrivacy.org.

### Some basic db setup



In [20]:
dbname = 'beforeiagree_db'
username = 'peterostendorp'

#Create engine
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://peterostendorp@localhost/beforeiagree_db


### Sites table setup
Read in sites, including those not covered by the OPP-115 corpus.

In [3]:
sites = pd.read_csv('../OPP-115/documentation/websites_covered_opp115.csv',index_col=3,parse_dates=[4])
sites.head()

,Site UID,Site URL,Site Human-Readable Name,Site Check Date,In 115 Set?,Comments,Sectoral Data,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106
Policy UID,,,,,,,,,,,,,,,,,,,,,
20,1,theatlantic.com,The Atlantic,2016-02-08,Yes,"Alexa Rank: 975 (Global), 289 (US)","Arts: Literature: Authors: A: Alcott, Louisa M...","Arts: Literature: Authors: M: Merwin, W. S.: W...",Arts: Literature: World Literature: American: ...,"Arts: Music: Composition: Composers: T: Toch, ...",...,Regional: North America: United States: Tennes...,Regional: North America: United States: Vermon...,Regional: North America: United States: Washin...,Regional: Oceania: Australia: New South Wales:...,Science: Agriculture: Crop Plants: Coffee,Science: Astronomy: Solar System: Dwarf Planet...,Science: Biology: Microbiology: Virology,Science: Earth Sciences: Atmospheric Sciences:...,Science: Social Sciences: Economics: Economic ...,Science: Social Sciences: Economics: People: G...
21,2,imdb.com,IMDb,2016-02-08,Yes,"Alexa Rank: 49 (Global), 27 (US)",Kids and Teens: Entertainment: Animation: Movies,Regional: Europe: United Kingdom: Arts and Ent...,Arts: Performing Arts: Magic: Magicians: Siegf...,Arts: Television: Programs: Science Fiction an...,...,"Arts: People: S: Sagal, Jean and Liz","Arts: People: W: Wilde, Lyn and Lee",Arts: Performing Arts: Magic: Magicians: Penn ...,Arts: Performing Arts: Puppetry: Muppets: Movi...,Arts: Performing Arts: Stunts: Players,Arts: Performing Arts: Theatre: Awards: Americ...,Arts: Television: Programs: Action and Adventu...,Arts: Television: Programs: Action and Adventu...,Arts: Television: Programs: Children's: Beetle...,Arts: Television: Programs: Comedy: Sitcoms: W...
26,3,nytimes.com,New York Times,2016-02-08,Yes,"Alexa Rank: 101 (Global), 22 (US)",Society: Issues: Warfare and Conflict: Specifi...,Science: Social Sciences: Psychology: Evolutio...,Society: Issues: Warfare and Conflict: Specifi...,Computers: E-Books: News and Media,...,Science: Technology: Space: Missions: Manned: ...,Society: Activism: Media: Culture Jamming: Spo...,Society: Crime: Sex Offenses: Child Pornograph...,Society: History: By Topic: Maritime: Ships: O...,"Society: History: Historians: Parker, Geoffrey",Society: Issues: Business: Allegedly Unethical...,Society: Issues: Education: Standardized Testing,Society: Issues: Environment: Light Pollution:...,Society: Issues: Environment: News and Media: ...,Society: Issues: Environment: News and Media: ...
32,4,theverge.com,The Verge,2016-02-08,Yes,"Alexa Rank: 525 (Global), 230 (US)",Home: Consumer Information: Computers and Inte...,Home: Consumer Information: Electronics: Weblogs,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,5,nbc.com,NBC Universal,2016-02-08,Yes,"Alexa Rank: 1548 (Global), 426 (US)",Kids and Teens: Entertainment: Television: Rea...,Arts: Television: Networks: NBC,Arts: Television: Programs: Action and Adventu...,Arts: Television: Programs: Comedy: Sitcoms: 3...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Change interpretation of 115 set membership column
sites['In 115 Set?']=sites['In 115 Set?'].apply(lambda yn: True if yn == 'Yes' else False)
sites.head()

,Site UID,Site URL,Site Human-Readable Name,Site Check Date,In 115 Set?,Comments,Sectoral Data,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106
Policy UID,,,,,,,,,,,,,,,,,,,,,
20,1,theatlantic.com,The Atlantic,2016-02-08,True,"Alexa Rank: 975 (Global), 289 (US)","Arts: Literature: Authors: A: Alcott, Louisa M...","Arts: Literature: Authors: M: Merwin, W. S.: W...",Arts: Literature: World Literature: American: ...,"Arts: Music: Composition: Composers: T: Toch, ...",...,Regional: North America: United States: Tennes...,Regional: North America: United States: Vermon...,Regional: North America: United States: Washin...,Regional: Oceania: Australia: New South Wales:...,Science: Agriculture: Crop Plants: Coffee,Science: Astronomy: Solar System: Dwarf Planet...,Science: Biology: Microbiology: Virology,Science: Earth Sciences: Atmospheric Sciences:...,Science: Social Sciences: Economics: Economic ...,Science: Social Sciences: Economics: People: G...
21,2,imdb.com,IMDb,2016-02-08,True,"Alexa Rank: 49 (Global), 27 (US)",Kids and Teens: Entertainment: Animation: Movies,Regional: Europe: United Kingdom: Arts and Ent...,Arts: Performing Arts: Magic: Magicians: Siegf...,Arts: Television: Programs: Science Fiction an...,...,"Arts: People: S: Sagal, Jean and Liz","Arts: People: W: Wilde, Lyn and Lee",Arts: Performing Arts: Magic: Magicians: Penn ...,Arts: Performing Arts: Puppetry: Muppets: Movi...,Arts: Performing Arts: Stunts: Players,Arts: Performing Arts: Theatre: Awards: Americ...,Arts: Television: Programs: Action and Adventu...,Arts: Television: Programs: Action and Adventu...,Arts: Television: Programs: Children's: Beetle...,Arts: Television: Programs: Comedy: Sitcoms: W...
26,3,nytimes.com,New York Times,2016-02-08,True,"Alexa Rank: 101 (Global), 22 (US)",Society: Issues: Warfare and Conflict: Specifi...,Science: Social Sciences: Psychology: Evolutio...,Society: Issues: Warfare and Conflict: Specifi...,Computers: E-Books: News and Media,...,Science: Technology: Space: Missions: Manned: ...,Society: Activism: Media: Culture Jamming: Spo...,Society: Crime: Sex Offenses: Child Pornograph...,Society: History: By Topic: Maritime: Ships: O...,"Society: History: Historians: Parker, Geoffrey",Society: Issues: Business: Allegedly Unethical...,Society: Issues: Education: Standardized Testing,Society: Issues: Environment: Light Pollution:...,Society: Issues: Environment: News and Media: ...,Society: Issues: Environment: News and Media: ...
32,4,theverge.com,The Verge,2016-02-08,True,"Alexa Rank: 525 (Global), 230 (US)",Home: Consumer Information: Computers and Inte...,Home: Consumer Information: Electronics: Weblogs,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,5,nbc.com,NBC Universal,2016-02-08,True,"Alexa Rank: 1548 (Global), 426 (US)",Kids and Teens: Entertainment: Television: Rea...,Arts: Television: Networks: NBC,Arts: Television: Programs: Action and Adventu...,Arts: Television: Programs: Comedy: Sitcoms: 3...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Reinterpret the categories for the sites. Find the primary category and take the mode across all columns
sectors = sites[sites.columns[6:]]
sectors = sectors.applymap(lambda s: str(s).split(':')[0])
sectors.head()

,Sectoral Data,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,...,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106
Policy UID,,,,,,,,,,,,,,,,,,,,,
20,Arts,Arts,Arts,Arts,Arts,Arts,Regional,Regional,Science,Society,...,Regional,Regional,Regional,Regional,Science,Science,Science,Science,Science,Science
21,Kids and Teens,Regional,Arts,Arts,Arts,Arts,Arts,Arts,Regional,Sports,...,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts
26,Society,Science,Society,Computers,Arts,Arts,Arts,Society,Regional,Society,...,Science,Society,Society,Society,Society,Society,Society,Society,Society,Society
32,Home,Home,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
33,Kids and Teens,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [6]:
s = []
for i in range(0,sectors.shape[0]):
    
    sec = sectors.iloc[i][sectors.iloc[i]!='nan'].mode()
    sec = sec.iloc[0] if len(sec) > 0 else 'None'
    s.append(sec)

In [7]:
s

['Arts',
 'Arts',
 'Arts',
 'Home',
 'Arts',
 'Arts',
 'Home',
 'Home',
 'Home',
 'Kids and Teens',
 'Regional',
 'Computers',
 'World',
 'Arts',
 'World',
 'News',
 'Sports',
 'Regional',
 'Arts',
 'Health',
 'Games',
 'Arts',
 'Games',
 'World',
 'Regional',
 'Business',
 'Business',
 'World',
 'Arts',
 'Regional',
 'Arts',
 'Games',
 'News',
 'Business',
 'Health',
 'Home',
 'Regional',
 'Reference',
 'Regional',
 'Regional',
 'Health',
 'Regional',
 'Business',
 'Regional',
 'Regional',
 'World',
 'Shopping',
 'Games',
 'Reference',
 'Business',
 'Business',
 'Regional',
 'Regional',
 'Recreation',
 'Games',
 'Home',
 'Home',
 'Kids and Teens',
 'Science',
 'Shopping',
 'Recreation',
 'Science',
 'Society',
 'Regional',
 'Health',
 'Health',
 'Health',
 'None',
 'Science',
 'Science',
 'Regional',
 'Computers',
 'Kids and Teens',
 'Games',
 'Science',
 'Science',
 'Home',
 'Regional',
 'Arts',
 'Kids and Teens',
 'Regional',
 'Recreation',
 'Regional',
 'Recreation',
 'Regional',
 

In [8]:
#Drop and append
sites = sites.drop(sites.columns[6:], axis=1)
sites['Sector'] = s

In [9]:
sites.head()

,Site UID,Site URL,Site Human-Readable Name,Site Check Date,In 115 Set?,Comments,Sector
Policy UID,,,,,,,
20,1,theatlantic.com,The Atlantic,2016-02-08,True,"Alexa Rank: 975 (Global), 289 (US)",Arts
21,2,imdb.com,IMDb,2016-02-08,True,"Alexa Rank: 49 (Global), 27 (US)",Arts
26,3,nytimes.com,New York Times,2016-02-08,True,"Alexa Rank: 101 (Global), 22 (US)",Arts
32,4,theverge.com,The Verge,2016-02-08,True,"Alexa Rank: 525 (Global), 230 (US)",Home
33,5,nbc.com,NBC Universal,2016-02-08,True,"Alexa Rank: 1548 (Global), 426 (US)",Arts


In [10]:
#Grab the policies table
policies = pd.read_csv('../OPP-115/documentation/policies_opp115.csv',index_col=0,parse_dates=[2,3])
policies = policies.drop('Unnamed: 4',axis=1)
policies.head()

,Policy URL,Policy collection date,Policy last updated date
Policy UID,,,
20,theatlantic.com/privacy-policy/,2015-07-02,2015-01-01
21,imdb.com/privacy,2015-07-02,2014-12-05
26,nytimes.com/privacy,2015-07-08,2015-06-10
32,voxmedia.com/privacy-policy,2015-07-02,2014-05-01
33,nbcuniversal.com/privacy/full-privacy-policy,2015-07-02,2015-01-14


In [11]:
#Grab the original policy text from html and append to this table
from os import listdir
import re
from lxml import etree, html
base_dir = '../OPP-115/sanitized_policies/'
files = [f for f in listdir(base_dir) if f.endswith('.html')]

#Check matching policy ids
# pids = sites.index.tolist()
# for f in files:
#     pid = np.int(f.split('_')[0])
#     if pid not in pids:
#         print('MISSING '+str(pid))

In [14]:
txt = []
pids = []
for f in files:
    with open(base_dir + f, 'r') as pg:
        #print('Reading: ' + f)
        page = pg.read()
        txt.append(page)
        pid = np.int(f.split('_')[0])
        pids.append(pid)

In [15]:
#Create dataframe of texts and join it to policies frame
txts = pd.DataFrame({'policy_text': txt},index=pids)
txts.index.name = 'Policy UID'
policies = pd.merge(policies,txts,left_index=True,right_index=True,how='outer')

In [16]:
policies.head()

,Policy URL,Policy collection date,Policy last updated date,policy_text
Policy UID,,,,
20,theatlantic.com/privacy-policy/,2015-07-02,2015-01-01,<strong> Privacy Policy </strong> <br> <br> <s...
21,imdb.com/privacy,2015-07-02,2014-12-05,"IMDb Privacy Notice <br> <br>|||Last Updated, ..."
26,nytimes.com/privacy,2015-07-08,2015-06-10,<strong> Privacy Policy </strong> <br> <br> La...
32,voxmedia.com/privacy-policy,2015-07-02,2014-05-01,Vox Media Privacy Policy <br> <br>|||<strong> ...
33,nbcuniversal.com/privacy/full-privacy-policy,2015-07-02,2015-01-14,Full Privacy Policy <br> <br> Last updated: 14...


In [17]:
#Now merge the sites and policies tables
sites = pd.merge(sites,policies,left_index=True,right_index=True,how='outer')
sites.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 191 entries, 20 to 1713
Data columns (total 11 columns):
Site UID                    191 non-null int64
Site URL                    191 non-null object
Site Human-Readable Name    191 non-null object
Site Check Date             191 non-null datetime64[ns]
In 115 Set?                 191 non-null bool
Comments                    191 non-null object
Sector                      191 non-null object
Policy URL                  191 non-null object
Policy collection date      191 non-null datetime64[ns]
Policy last updated date    159 non-null datetime64[ns]
policy_text                 191 non-null object
dtypes: bool(1), datetime64[ns](3), int64(1), object(6)
memory usage: 16.6+ KB


In [18]:
sites.index.nunique()

115

In [21]:
#Persist to sites table in beforeiagree_db
sites.to_sql('sites', engine, if_exists='replace')

### Annotations table

Each CSV file in annotations/ contains the annotation data for one privacy policy. Each policy has been annotated by three workers, and data from all three workers is present in the same file. These files do not represent any efforts to consolidate information from multiple annotators.

These CSV files use the Excel dialect. Each row contains the information for a single data practice. The columns are:
1. annotation ID (a globally unique identifier for a data practice)
2. batch ID (name of a batch in the annotation tool; often indicates who the annotators were)
3. annotator ID
4. policy ID (this corresponds to the numeric prefixes in the policy filename, as found in other directories)
5. segment ID (the zero-indexed, sequential identifier of the policy segment; e.g., the first segment in a policy's text is segment zero)
6. category name
7. attribute-value pairs (represented as JSON)
8. policy URL
9. date

The attribute-value dictionary (column 7/G), at its highest level, consists of keys (strings that correspond to attribute names) that map to nested dictionaries. The nested dictionaries have keys that specify the selected text, its location in the segment, and the value associated with the attribute.

**Consolidation** - In addition, we're using the consolidated annotations at threshold value of 0.5. Additional guidance on annotation consolidation:

Since multiple annotators worked on each privacy policy, they produced some redundant annotations. Each of the three subdirectories in consolidation/ contains the results of the consolidation algorithm with a different convergence threshold, as described in the paper. The annotations are in CSV files in the same format as described in the _Annotations_ section of this document.

In the consolidation CSV files, the data practices that were created by consolidation (i.e., by merging together 2-3 data practices, each from a different annotator) have a 'C' prefix at the beginning of their annotation IDs. All others are "singlet" data practices, identified by one annotator and too distinct for consolidation.

In [76]:
#Locate the files
from os import listdir
import re
ann_dir = '../OPP-115/annotations/'
files = [f for f in listdir(ann_dir) if f.endswith('.csv')]

In [77]:
#Set up name conventions and data conversions for import
import numpy as np
names = """annotation_id 
        batch_id 
        annotator_id 
        policy_id 
        segment_id 
        category_name 
        attributes_value_pairs 
        date 
        policy_url""".split()

types = {'annotation_id': np.int,
        'batch_id': str,
        'annotator_id': np.int,
        'policy_id': np.int,
        'segment_id': np.int,
        'category_name': str}

ann_list = []
pids = []
for f in files:
    df = pd.read_csv(ann_dir + f,header=None,names=names,na_values={'date': 'Not specified'},
                 parse_dates=[7],index_col=0)
    ann_list.append(df)
    pids.append(np.int(f.split('_')[0]))

In [81]:
annotations = pd.concat(ann_list,axis=0,keys=pids,names=['Policy UID','annotation_id'])
annotations = annotations.drop('policy_id',axis=1)

In [84]:
annotations.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 23194 entries, (1017, 20137) to (995, 6754)
Data columns (total 7 columns):
batch_id                  23194 non-null object
annotator_id              23194 non-null int64
segment_id                23194 non-null int64
category_name             23194 non-null object
attributes_value_pairs    23194 non-null object
date                      19460 non-null datetime64[ns]
policy_url                23194 non-null object
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 1.5+ MB


In [85]:
attr_values = pd.DataFrame(data=None,columns=['annotation_id','start_idx','end_idx','attribute','text','value'])
template = dict.fromkeys('startIndexInSegment endIndexInSegment selectedText value'.split())

import json
import time

In [97]:
#BATCH PROCESSING OF ATTRIBUTE-VALUE PAIRS: THIS NEEDS TO BE DONE MANUALLY!
attr_values1 = pd.DataFrame(data=None,columns=['annotation_id','start_idx','end_idx','attribute','text','value'])
t0 = time.time()
print('Starting at ' + str(t0))
for i in range(22001,annotations.shape[0]):    #Note manual adjustment of batch size here...
    
    attr_val = annotations['attributes_value_pairs'].iloc[i]
    ann_id = annotations.index.values[i][-1]
    
    obj = json.loads(attr_val)
    keys = list(obj.keys())
    
    for k in keys:
        obj2 = dict(template, **obj[k]) #Ensures at a minimum we get the empty template data.
        df = pd.DataFrame({'annotation_id': [ann_id],
                           'start_idx': [obj2['startIndexInSegment']],
                           'end_idx': [obj2['endIndexInSegment']],
                           'attribute': [k],
                           'text': [obj2['selectedText']],
                           'value': [obj2['value']]})
        attr_values1 = attr_values1.append(df,ignore_index=True)
t1 = time.time()
print('Finished at ' + str(t1))
print('Total elapsed time ' + str(t1-t0))
attr_values = attr_values.append(attr_values1,ignore_index=True)
attr_values.info()

Starting at 1505176729.055625
Finished at 1505176763.0873868
Total elapsed time 34.03176188468933
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146865 entries, 0 to 146864
Data columns (total 6 columns):
annotation_id    146865 non-null object
attribute        146865 non-null object
end_idx          146865 non-null object
start_idx        146865 non-null object
text             102640 non-null object
value            146865 non-null object
dtypes: object(6)
memory usage: 6.7+ MB


In [98]:
#Set up indices
attr_values.index.name = 'text_selection_id'
attr_values = attr_values.set_index(['annotation_id'],
                                    append=True).reorder_levels(['annotation_id','text_selection_id'])

In [99]:
attr_values = attr_values.rename(columns={'text':'text_selection'})

In [112]:
attr_values['text_selection'][(attr_values['text_selection'].isnull()) | (attr_values['text_selection'] == 'null')] = 'Not selected'
attr_values['text_selection']

annotation_id  text_selection_id
20137          0                    Sci-News.com is committed to protecting and re...
20324          1                    nformation that you provide by filling in form...
               2                                                         Not selected
               3                                 Sci-News.com may collect and process
               4                                                            including
               5                                                         Not selected
               6                                                         Not selected
               7                                                         Not selected
               8                                                         Not selected
               9                    other purposes, for example when you report a ...
20325          10                   nformation that you provide by filling in form...
               11    

In [113]:
#Persist all of this crap to a new table. Phew!
annotations.to_sql('annotations', engine, if_exists='replace')
attr_values.to_sql('text_selections_new', engine, if_exists='replace')

### Pretty annotations table

In [66]:
pretty_dir = '../OPP-115/pretty_print/'
files = [f for f in listdir(pretty_dir) if f.endswith('.csv')]

In [72]:
test = pd.read_csv(pretty_dir + files[0],header=None,names=['annotation_id','segment_id',
                                                            'annotator_id','pretty_string'])
test = test.set_index(['annotation_id'],append=False)

In [73]:
names = """annotation_id 
        segment_id 
        annotator_id 
        pretty_string""".split()

types = {'annotation_id': np.int,
        'segment_id': np.int,
        'annotator_id': np.int,
        'pretty_string': str}

pretty_list = []
for f in files:
    df = pd.read_csv(pretty_dir + f,header=None,names=names,index_col=0,dtype=types)
    pretty_list.append(df)

In [74]:
pretty = pd.concat(pretty_list)
pretty.head()

,segment_id,annotator_id,pretty_string
annotation_id,,,
275,0,84,"The text introduces the policy, a section, or ..."
619,0,82,"The text introduces the policy, a section, or ..."
1332,0,88,"The text introduces the policy, a section, or ..."
276,1,84,"The policy makes generic security statements, ..."
620,1,82,"The text introduces the policy, a section, or ..."


In [75]:
pretty.to_sql('pretty_strings', engine, if_exists='replace')

## Segments table

This table contains a fully indexed list of all segments so that they can be mapped back to annotations and classifications. This is the crux of the data structure that ties expert annotations back to the original policy text.

In [36]:
sites.head()

,Site UID,Site URL,Site Human-Readable Name,Site Check Date,In 115 Set?,Comments,Sector,Policy URL,Policy collection date,Policy last updated date,policy_text
Policy UID,,,,,,,,,,,
20,1,theatlantic.com,The Atlantic,2016-02-08,True,"Alexa Rank: 975 (Global), 289 (US)",Arts,theatlantic.com/privacy-policy/,2015-07-02,2015-01-01,<strong> Privacy Policy </strong> <br> <br> <s...
21,2,imdb.com,IMDb,2016-02-08,True,"Alexa Rank: 49 (Global), 27 (US)",Arts,imdb.com/privacy,2015-07-02,2014-12-05,"IMDb Privacy Notice <br> <br>|||Last Updated, ..."
26,3,nytimes.com,New York Times,2016-02-08,True,"Alexa Rank: 101 (Global), 22 (US)",Arts,nytimes.com/privacy,2015-07-08,2015-06-10,<strong> Privacy Policy </strong> <br> <br> La...
32,4,theverge.com,The Verge,2016-02-08,True,"Alexa Rank: 525 (Global), 230 (US)",Home,voxmedia.com/privacy-policy,2015-07-02,2014-05-01,Vox Media Privacy Policy <br> <br>|||<strong> ...
32,1000,voxmedia.com,Vox Media,2015-02-14,False,"Alexa Rank: 40248 (Global), 18551 (US)",None,voxmedia.com/privacy-policy,2015-07-02,2014-05-01,Vox Media Privacy Policy <br> <br>|||<strong> ...


In [38]:
#First grab policy text from the sites table to parse down
policies = pd.DataFrame(sites['policy_text'])
policies.head()

,policy_text
Policy UID,
20,<strong> Privacy Policy </strong> <br> <br> <s...
21,"IMDb Privacy Notice <br> <br>|||Last Updated, ..."
26,<strong> Privacy Policy </strong> <br> <br> La...
32,Vox Media Privacy Policy <br> <br>|||<strong> ...
32,Vox Media Privacy Policy <br> <br>|||<strong> ...


In [73]:
#For each policy, create a dataframe of segments with unique segment index
dfs = []
for i in range(0,policies.shape[0]):
    tmp = pd.DataFrame(policies['policy_text'].iloc[i].split('|||'),columns=['segments'])
    tmp['Policy UID'] = policies.index.values[i]
    tmp.index.name = 'segment_id'
    dfs.append(tmp)

,segments,Policy UID
segment_id,,
0,<strong> Privacy Policy </strong> <br> <br> <s...,20
1,This privacy policy does not apply to Sites ma...,20
2,"By visiting our Sites, you are accepting the p...",20
3,<strong> What Information Is Collected? </stro...,20
4,<strong> Personally Identifiable Information <...,20
5,<strong> Non-Personally Identifiable Informati...,20
6,<strong> Collection of Personally Identifiable...,20
7,<strong> Collection of Your Source IP Address/...,20
8,<strong> How Do the Sites Use Cookies and Othe...,20


In [76]:
segments = pd.concat(dfs,axis=0).reset_index().set_index(['Policy UID','segment_id'])

In [77]:
#Create table
segments.to_sql('segments', engine, if_exists='replace')